<a href="https://colab.research.google.com/github/Smart-Pigs/Depth-Images/blob/main/WE_EfficientNetB2_TransferLearning_CroppedImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure you have this folder as shortcut in your drive - /content/gdrive/MyDrive/Morota-video-data/ 

In [1]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


### Data Loading

In [2]:
#@markdown Select resolution of input dataset and weeks of data

Instantiate_New_Dataset = True #@param {type:"boolean"}
Refresh_Old_Dataset = False #@param {type:"boolean"}

if Instantiate_New_Dataset == True:
  !rm -rf Week*/

Resolution = "256"  #@param ['720', '512', '256']

if Refresh_Old_Dataset == False:
  if Resolution == "720":
    !cp /content/gdrive/MyDrive/SmartPig/References/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_720.zip .
    !unzip -qq CROPPED_WEEK1-10_720.zip -d .
    !rm -rf *WEEK*.zip

  if Resolution == "512":
    !cp /content/gdrive/MyDrive/SmartPig/References/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_512.zip .
    !unzip -qq CROPPED_WEEK1-10_512.zip -d .
    !rm -rf *WEEK*.zip

  if Resolution == "256":
    !cp /content/gdrive/MyDrive/Morota-video-data/DEPTH_INDIVIDUAL/CROPPED/CROPPED_WEEK1-10_256.zip .
    !unzip -qq CROPPED_WEEK1-10_256.zip -d .
    !rm -rf *WEEK*.zip

### Mapping Table Input

In [3]:
import pandas as pd

In [4]:
final_map = pd.read_csv("/content/gdrive/MyDrive/Morota-video-data/final_mapping.csv")

In [5]:
weeks = final_map

In [6]:
weeks

,image_path,Date,Pig,weight,Week
0,Week1/20210922/20210922_3330_b/_Depth_4683.jpg,20210922,3330,15.5,1
1,Week1/20210922/20210922_3330_b/_Depth_4778.jpg,20210922,3330,15.5,1
2,Week1/20210922/20210922_3330_b/_Depth_4579.jpg,20210922,3330,15.5,1
3,Week1/20210922/20210922_3330_b/_Depth_4578.jpg,20210922,3330,15.5,1
4,Week1/20210922/20210922_3330_b/_Depth_4584.jpg,20210922,3330,15.5,1
...,...,...,...,...,...
96220,Week10/20211119/20211119_3342/_Depth_3504.jpg,20211119,3342,56.6,10
96221,Week10/20211119/20211119_3342/_Depth_3318.jpg,20211119,3342,56.6,10
96222,Week10/20211119/20211119_3342/_Depth_3476.jpg,20211119,3342,56.6,10
96223,Week10/20211119/20211119_3342/_Depth_3169.jpg,20211119,3342,56.6,10


In [7]:
weeks_train = weeks[weeks["Pig"] != 3384]

In [8]:
weeks_test = weeks[weeks["Pig"] == 3384]

In [9]:
image_paths_train = weeks_train['image_path'].values
labels_train = weeks_train['weight'].values
image_paths_test = weeks_test['image_path'].values
labels_test = weeks_test['weight'].values

### Dataset

In [10]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 18.9 MB/s eta 0:00:00


In [11]:
import tensorflow as tf
import keras
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import EfficientNetB2
from tensorflow_addons.metrics import RSquare

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((image_paths_train, labels_train))
test_dataset = tf.data.Dataset.from_tensor_slices((image_paths_test, labels_test))

In [13]:
IMG_SIZE = int(Resolution)
channels = 3

def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    #img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.rgb_to_grayscale(img)
    if int(Resolution) > IMG_SIZE:
      img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

In [14]:
print("Number of elements in the train dataset:", tf.data.experimental.cardinality(train_dataset).numpy())
print("Number of elements in the test dataset:", tf.data.experimental.cardinality(test_dataset).numpy())

Number of elements in the train dataset: 77871
Number of elements in the test dataset: 18354


In [15]:
batch_size = 128

train_dataset = train_dataset.shuffle(len(labels_train))
train_dataset = train_dataset.map(preprocess_image)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(preprocess_image)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

### Model Config and Training

In [16]:
from keras.layers.normalization.batch_normalization import BatchNormalization
base_model = EfficientNetB2(weights='imagenet', 
                            input_tensor=None,
                            input_shape=None,
                            include_top = False);
base_model.trainable = False;
for layer in base_model.layers[-18:]:
  if not isinstance(layer, BatchNormalization):
    layer.trainable = True;

31790344/31790344 [==============================] - 2s 0us/step


In [17]:
from keras.layers.pooling.global_average_pooling2d import GlobalAveragePooling2D
def head(base_output):
  # x_input = tf.keras.layers.Input(shape)
  x= GlobalAveragePooling2D()(base_output)
  x= BatchNormalization()(x)

  x= tf.keras.layers.Dense(units = 512, kernel_regularizer= 'l2',kernel_initializer='he_uniform')(x)
  x= BatchNormalization()(x)
  x= tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)

  x= tf.keras.layers.Dense(units = 512, kernel_regularizer= 'l2',kernel_initializer='he_uniform')(x)
  x= BatchNormalization()(x)
  x= tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)

  x= tf.keras.layers.Dense(units = 1024, kernel_regularizer= 'l2',kernel_initializer='he_uniform')(x)
  x= BatchNormalization()(x)
  x= tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)

  x= tf.keras.layers.Dense(units = 512, kernel_regularizer= 'l2',kernel_initializer='he_uniform')(x)
  x= BatchNormalization()(x)
  x= tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)

  x = tf.keras.layers.Dense(1)(x);
  # model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
  
  return x


In [18]:
model= tf.keras.models.Model(base_model.input, head(base_model.output), name='transferlearning')

In [19]:
# tfa.metrics.RSquare(name="r2_s", dtype=None)

In [20]:
model.compile(optimizer='adam', loss='mse', metrics=['mae','mape'])

In [21]:
model.summary()

Model: "transferlearning"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling (Rescaling)          (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 normalization (Normalization)  (None, None, None,   7           ['rescaling[0][0]']              
                                3)                                                 

In [22]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
609/609 [==============================] - 399s 593ms/step - loss: 226.8822 - mae: 9.5518 - mape: 30.3140 - val_loss: 394.2545 - val_mae: 18.6328 - val_mape: 77.0167
Epoch 2/10
609/609 [==============================] - 359s 590ms/step - loss: 104.9049 - mae: 7.1344 - mape: 23.9988 - val_loss: 335.8233 - val_mae: 17.6331 - val_mape: 72.2440
Epoch 3/10
609/609 [==============================] - 360s 591ms/step - loss: 89.2684 - mae: 6.8907 - mape: 23.2204 - val_loss: 28.8803 - val_mae: 3.8172 - val_mape: 15.3880
Epoch 4/10
609/609 [==============================] - 360s 591ms/step - loss: 85.0002 - mae: 6.7859 - mape: 22.8815 - val_loss: 1079.9011 - val_mae: 32.5806 - val_mape: 132.6643
Epoch 5/10
609/609 [==============================] - 360s 591ms/step - loss: 82.6754 - mae: 6.6931 - mape: 22.6166 - val_loss: 576.0841 - val_mae: 23.6068 - val_mape: 96.7962
Epoch 6/10
609/609 [==============================] - 332s 544ms/step - loss: 81.2099 - mae: 6.6161 - mape: 22.3514 - 

In [23]:
test_loss, test_mae, test_mape = model.evaluate(test_dataset)
print('Test loss:', test_loss)
print('Test mean absolute error:', test_mae)
print('Test mean absolute percentage error:', test_mape)

144/144 [==============================] - 55s 378ms/step - loss: 162.0912 - mae: 12.0821 - mape: 49.5024
Test loss: 162.0911865234375
Test mean absolute error: 12.082135200500488
Test mean absolute percentage error: 49.50236129760742


Fine-tuning the model by unfreezing parts of base_model and retraining with low learning rate

In [ ]:
# batch_size = 64
for layers in base_model.layers[-20:]:
  layers.trainable = True


model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='mse', metrics=['mae','mape'])
model.fit(train_dataset, epochs=10, validation_data=test_dataset)


Epoch 1/10
609/609 [==============================] - 385s 612ms/step - loss: 85.8358 - mae: 6.8654 - mape: 22.6898 - val_loss: 302.7683 - val_mae: 16.9054 - val_mape: 68.8339
Epoch 2/10
609/609 [==============================] - 371s 608ms/step - loss: 72.6131 - mae: 6.2303 - mape: 20.9767 - val_loss: 205.3776 - val_mae: 13.6532 - val_mape: 55.1876
Epoch 3/10
609/609 [==============================] - 371s 609ms/step - loss: 70.2696 - mae: 6.1185 - mape: 20.7188 - val_loss: 181.5870 - val_mae: 12.7104 - val_mape: 51.3036
Epoch 4/10
390/609 [==================>...........] - ETA: 1:43 - loss: 67.9919 - mae: 5.9842 - mape: 20.3018